In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob

from tensorflow import keras

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras import regularizers

# import tensorflow as tf
import cv2
import random
import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
input_dir = '../'
limit = 200
cover_imgs = sorted(glob.glob(input_dir+'Cover/*.jpg'))
cover_imgs = cover_imgs[:limit]

jmipod_imgs = sorted(glob.glob(input_dir+'JMiPOD/*.jpg'))
jmipod_imgs = jmipod_imgs[:limit]#[-limit:]

juniward_imgs = sorted(glob.glob(input_dir+'JUNIWARD/*.jpg'))
juniward_imgs = juniward_imgs[:limit]#[-limit:]

uerd_imgs = sorted(glob.glob(input_dir+'UERD/*.jpg'))
uerd_imgs = uerd_imgs[:limit]#[-limit:]

In [ ]:
index = 2
print(cover_imgs[index])
print(jmipod_imgs[index])
print(juniward_imgs[index])
print(uerd_imgs[index])

In [ ]:
# list_IDs = cover_imgs + jmipod_imgs + juniward_imgs + uerd_imgs
# random.Random(4).shuffle(list_IDs)

till = int(0.8*len(cover_imgs))
print(till)

# train_IDs = []
# test_IDs = []

train_IDs = cover_imgs[:till]+juniward_imgs[:till]+uerd_imgs[:till]+jmipod_imgs[:till]
# train_IDs = cover_imgs[:100] + juniward_imgs[100:100*2]# + uerd_imgs[:till]
# train_IDs = [i.split('/')[-1] for i in train_IDs]
test_IDs = cover_imgs[till:]+juniward_imgs[till:]+uerd_imgs[till:]+jmipod_imgs[till:]
# test_IDs = cover_imgs[:50] + juniward_imgs[150:200]

In [ ]:
len(train_IDs)

In [ ]:
# def read_DCT(file):
#     c_struct=jio.read( file )
#     coverDCT = np.zeros([512,512,3]) ; coverDCT[:,:,0] = c_struct.coef_arrays[0] ; coverDCT[:,:,1] = c_struct.coef_arrays[1] ; coverDCT[:,:,2] = c_struct.coef_arrays[2]
#     return coverDCT

# # def get_image_data(image_id, image_size):
# #     img = cv2.imread(image_id)
# # #     h, w = image_size
# # #     img= img[100:100+h, 100:100+w]
# #     return cv2.resize(img, image_size)

# def get_image_data(image_id, image_size, COV_DIR='/kaggle/input/alaska2-image-steganalysis/Cover/', STEG_DIR='/kaggle/input/alaska2-image-steganalysis/JUNIWARD/'):
#     img_cov = cv2.imread(COV_DIR+image_id)
#     img_stego = cv2.imread(STEG_DIR+image_id)

# #     h, w = image_size
# #     img= img[100:100+h, 100:100+w]
#     return cv2.resize(img_cov, image_size), cv2.resize(img_stego, image_size)


# def get_label(image_id):
#     # 0 for cover, 1 for steg
#     return 0, 1
# #     if '/kaggle/input/alaska2-image-steganalysis/Cover/' in image_id:
# #         return 0
# #     else:
# #         return 1

    
# class DataGenerator(keras.utils.Sequence):
#     'Generates data for Keras'
#     def __init__(self, list_IDs, batch_size=32, dim=(32,32,32), n_channels=1, 
#                  shuffle=True, number_of_labels=2):
#         'Initialization'
#         self.dim = dim
#         self.batch_size = batch_size
#         self.list_IDs = list_IDs
#         self.n_channels = n_channels
#         self.shuffle = shuffle
#         self.on_epoch_end()
#         self.number_of_labels = number_of_labels
        
#     def __len__(self):
#         'Denotes the number of batches per epoch'
#         return int(np.floor(len(self.list_IDs) / self.batch_size))
    
#     def __getitem__(self, index):
#         'Generate one batch of data'
#         # Generate indexes of the batch
#         indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

#         # Find list of IDs
#         list_IDs_temp = [self.list_IDs[k] for k in indexes]

#         # Generate data
#         X, y = self.__data_generation(list_IDs_temp)
#         # cv2 recevrses height and width
#         return X, y

#     def on_epoch_end(self):
#         'Updates indexes after each epoch'
#         self.indexes = np.arange(len(self.list_IDs))
#         if self.shuffle == True:
#             np.random.shuffle(self.indexes)

#     def __data_generation(self, list_IDs_temp):
#         'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
#         # Initialization
# #         X = np.empty((self.batch_size, *self.dim[::-1], self.n_channels))
# #         y = np.empty((self.batch_size, *self.dim[::-1]))
#         X = np.empty((self.batch_size*2, *self.dim, self.n_channels), dtype=np.float32)
#         y = np.empty((self.batch_size*2), dtype=int)
        
#         # Generate data
#         for i, ID in enumerate(list_IDs_temp):
#             # Store sample
#             img_cov, img_stego = get_image_data(ID, image_size) #read_DCT(ID)
#             X[2*i,], X[2*i+1,] = img_cov, img_stego#changeColorSpace(img, flag_YCR_CB)[:, :, 0].reshape(*image_size,1)
#             y[2*i], y[2*i+1] = get_label(ID)
            
        
#         return X, y

In [ ]:
def read_DCT(file):
    c_struct=jio.read( file )
    coverDCT = np.zeros([512,512,3]) ; coverDCT[:,:,0] = c_struct.coef_arrays[0] ; coverDCT[:,:,1] = c_struct.coef_arrays[1] ; coverDCT[:,:,2] = c_struct.coef_arrays[2]
    return coverDCT

def get_image_data(image_id, image_size):
    img = cv2.imread(image_id)
#     h, w = image_size
#     img= img[100:100+h, 100:100+w]
    return cv2.resize(img, image_size)

def get_label(image_id):
    if '/kaggle/input/alaska2-image-steganalysis/Cover/' in image_id:
        return 0
    else:
        return 1

    
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(32,32,32), n_channels=1, 
                 shuffle=True, number_of_labels=2):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()
        self.number_of_labels = number_of_labels
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        # cv2 recevrses height and width
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
#         X = np.empty((self.batch_size, *self.dim[::-1], self.n_channels))
#         y = np.empty((self.batch_size, *self.dim[::-1]))
        X = np.empty((self.batch_size, *self.dim, self.n_channels), dtype=np.float)
        y = np.empty((self.batch_size), dtype=int)
        
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img = get_image_data(ID, image_size) #read_DCT(ID)
            X[i,] = img/255.#changeColorSpace(img, flag_YCR_CB)[:, :, 0].reshape(*image_size,1)
            y[i] = get_label(ID)
            
        
        return X, y

In [ ]:
image_size = (256, 256)
# Parameters
params = {'dim': image_size,
          'batch_size': 8,
          'n_channels': 3,
          'shuffle': True}

# Generators
training_generator = DataGenerator(train_IDs, **params)
validation_generator = DataGenerator(test_IDs, **params)

In [ ]:
X, y = training_generator.__getitem__(1)

y

In [ ]:
np.max(X[0] - X[1])

In [ ]:
plt.imshow(X[1])#.reshape((*image_size))/np.max(X[0]))

In [ ]:
X[2].shape

# SRNet

In [ ]:
import tensorflow as tf
# from tensorflow import keras

from tensorflow.keras.models import *

from tensorflow.keras import layers

In [ ]:
def SRNet(_inputs):
    penalty = 2e-4
    mom = 0.99
    reduction_axis = [1,2]
    #'Layer1' 
    conv=layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
           kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(_inputs) 
    actv=tf.nn.relu(layers.BatchNormalization(momentum=mom)(conv))
    # Layer2'
    conv=layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                      kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(actv)
    actv=tf.nn.relu(layers.BatchNormalization(momentum=mom)(conv))
    #Layer3'
    conv1=layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(actv)
    actv1=tf.nn.relu(layers.BatchNormalization(momentum=mom)(conv1))
    conv2=layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(actv1)
    bn2=layers.BatchNormalization(momentum=mom)(conv2)
    res= tf.add(actv, bn2)
    # Layer4'
    conv1=layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(res, )
    actv1=tf.nn.relu(layers.BatchNormalization(momentum=mom)(conv1))
    conv2=layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(actv1)
    bn2=layers.BatchNormalization(momentum=mom)(conv2)
    res= tf.add(res, bn2)
    # Layer5
    conv1=layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(res)
    actv1=tf.nn.relu(layers.BatchNormalization(momentum=mom)(conv1))
    conv2=layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(actv1)
    bn=layers.BatchNormalization(momentum=mom)(conv2)
    res= tf.add(res, bn)
    # 'Layer6
    conv1=layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(res)
    actv1=tf.nn.relu(layers.BatchNormalization(momentum=mom)(conv1))
    conv2=layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(actv1)
    bn=layers.BatchNormalization(momentum=mom)(conv2)
    res= tf.add(res, bn)
    # Layer7'
    conv1=layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(res)
    actv1=tf.nn.relu(layers.BatchNormalization(momentum=mom)(conv1))
    conv2=layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(actv1)
    bn=layers.BatchNormalization(momentum=mom)(conv2)
    res= tf.add(res, bn)
    # with tf.variable_scope('Layer8'): 
    convs = layers.Conv2D(filters=16,  kernel_size=1, strides=2)(res)
    convs = layers.BatchNormalization(momentum=mom)(convs)
    conv1 = layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                         kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(res)
    actv1=tf.nn.relu(layers.BatchNormalization(momentum=mom)(conv1))
    conv2=layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(actv1)
    bn=layers.BatchNormalization(momentum=mom)(conv2)
    pool = tf.nn.avg_pool2d(bn, ksize=[3,3], strides=[2,2], padding='SAME')
    res= tf.add(convs, pool)
    #             with tf.variable_scope('Layer9'):  
    convs = layers.Conv2D(filters=64, kernel_size=1, strides=2)(res)
    convs = layers.BatchNormalization(momentum=mom)(convs)
    conv1=layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(res)
    actv1=tf.nn.relu(layers.BatchNormalization(momentum=mom)(conv1))
    conv2=layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(actv1)
    bn=layers.BatchNormalization(momentum=mom)(conv2)
    pool = tf.nn.avg_pool2d(bn, ksize=[3,3], strides=[2,2], padding='SAME')
    res= tf.add(convs, pool)
    # with tf.variable_scope('Layer10'): 
    convs = layers.Conv2D(filters=128, kernel_size=1, strides=2, padding='SAME', kernel_initializer='he_uniform',
                         kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(res)
    convs = layers.BatchNormalization(momentum=mom)(convs)
    conv1=layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(res)
    actv1=tf.nn.relu(layers.BatchNormalization(momentum=mom)(conv1))
    conv2=layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(actv1)
    bn=layers.BatchNormalization(momentum=mom)(conv2)
    pool = tf.nn.avg_pool2d(bn, ksize=[3,3], strides=[2,2], padding='SAME')
    res= tf.add(convs, pool)
    # with tf.variable_scope('Layer11'): 
    convs = layers.Conv2D(filters=256, kernel_size=1, strides=2, padding='SAME', kernel_initializer='he_uniform',
                         kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(res)
    convs = layers.BatchNormalization(momentum=mom)(convs)
    conv1=layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(res)
    actv1=tf.nn.relu(layers.BatchNormalization(momentum=mom)(conv1))
    conv2=layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(actv1)
    bn=layers.BatchNormalization(momentum=mom)(conv2)
    pool = tf.nn.avg_pool2d(bn, ksize=[3,3], strides=[2,2], padding='SAME')
    res= tf.add(convs, pool)
    # with tf.variable_scope('Layer12'): 
    conv1=layers.Conv2D(filters=512, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(res)
    actv1=tf.nn.relu(layers.BatchNormalization(momentum=mom)(conv1))
    conv2=layers.Conv2D(filters=512, kernel_size=3, strides=1, padding='SAME', kernel_initializer='he_uniform',
                       kernel_regularizer=tf.keras.regularizers.l2(penalty), bias_initializer=tf.constant_initializer(0.2))(actv1)
    bn=layers.BatchNormalization(momentum=mom)(conv2)
    avgp = tf.reduce_mean(bn, reduction_axis,  keepdims=True )
    
    fc1 = layers.Flatten()(avgp)
#     do1 = layers.Dropout(0.2)(fc1)
    d3 = layers.Dense(1, activation='sigmoid')(fc1)
    
    return d3

In [ ]:
input_img = layers.Input(shape = (*image_size, 3))

# input_img = tf.cast(tf.transpose(inputs, [0, 3, 1, 2]), tf.float32)

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

sr_ = Model(input_img, SRNet(input_img))
# sr_ = tf.keras.models.load_model('/kaggle/working/SRNet_256_256_v1.h5')

sr_.compile(loss=keras.losses.binary_crossentropy, 
                    metrics=METRICS, optimizer=tf.keras.optimizers.Adamax(learning_rate=1e-3))


In [ ]:
# del sr_

In [ ]:
sr_ = tf.keras.models.load_model('SRNet_256_256_v2.h5')

In [ ]:
# sr_.evaluate(training_generator, verbose=2)
# sr_.metrics

In [ ]:
def scheduler(epoch):
    if epoch < 25:
        return 1e-3
    else:
        return 1e-4# * tf.math.exp(0.1 * (10 - epoch))
    
checkpoint_filepath = '/kaggle/working'
filepath=checkpoint_filepath+"/model-{epoch:02d}-{val_accuracy:.2f}.hdf5"

    
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    monitor='val_accuracy', verbose=1,
    save_best_only=False, save_weights_only=False, 
    save_frequency=10)

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
sr_model_history = sr_.fit_generator(generator=training_generator, 
                                     validation_data=validation_generator, epochs=10)#, callbacks=[callback])

In [ ]:
# import h5py

In [ ]:
# ! rm /kaggle/working/model*

In [ ]:
sr_.save('/kaggle/working/SRNet_256_256_v2.h5')
# tf.keras.models.save_model(
#     sr_, 'SRNet_256_256_v1.h5', overwrite=True, include_optimizer=True, save_format='h5',
#     signatures=None, options=None
# )

In [ ]:
# sr_model_history.history['loss']

In [ ]:
# summarize history for accuracy
plt.plot(sr_model_history.history['accuracy'])
plt.plot(sr_model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(sr_model_history.history['auc'])
plt.plot(sr_model_history.history['val_auc'])
plt.title('model auc')
plt.ylabel('auc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(sr_model_history.history['loss'])
plt.plot(sr_model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [2]:
import numpy as np

In [33]:
0.1 * (10 - i//25)

0.7000000000000001

In [47]:
lr = 0.001

for i in range(1, 20):
    if i%10 == 0:
        lr/=10
    print(lr)

0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.001
0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
0.0001
